# 不等式のQUBO変換を用いて基底エネルギー以下のサンプルを得る

### import

In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
import random

### 各変数

In [3]:
X = pd.read_csv('../input/geneX_df.txt',sep ='\t')# 説明変数
print("X.shape is {}".format(X.shape))
X.head()

X.shape is (781, 16799)


,ARHGEF10L,HIF3A,RNF17,RNF11,REM1,RTN4RL2,C16orf13,C16orf11,FGFR1OP2,TSKS,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
TCGA-A8-A092-01,-0.92667,-4.836158,-0.545559,-0.425362,-2.511796,1.420118,-0.076931,-0.583953,-0.064954,-1.262546,...,-0.727478,-1.319172,-0.534283,-0.871063,0.137041,-0.494064,-0.396097,-1.178462,-1.748619,-0.659749
TCGA-D8-A1JK-01,-0.83967,-3.260158,0.420041,0.718538,-0.322796,-1.267982,0.252869,-0.583953,-0.031154,-1.634746,...,-0.165278,-1.691372,0.783017,0.037337,-0.051459,0.730136,1.388803,1.339138,-1.924519,0.075451
TCGA-E2-A10C-01,-0.11937,-4.646958,-0.545559,-0.453962,-1.073796,-1.506382,1.084969,1.680847,0.010146,0.890654,...,-0.166078,-1.129972,-0.350883,-0.681864,-0.206159,-0.406364,0.354103,-0.003162,-0.903719,-0.195349
TCGA-BH-A0C7-01,0.28803,-1.599358,-0.545559,-0.211162,-2.041396,-0.156182,-0.861231,-0.583953,-0.249454,-1.029746,...,-0.122478,-0.488172,-0.616583,-1.243263,0.330741,-0.188764,0.420403,-0.456262,0.914881,-0.427849
TCGA-A8-A06N-01,0.61053,-4.268858,-0.545559,0.764638,-2.182996,-1.431682,-0.876131,-0.583953,0.560146,-1.634746,...,-0.429378,2.566228,-1.073483,-1.243263,0.133141,-2.727864,-0.306397,-0.882762,-3.308419,-0.224949


In [4]:
y_df = pd.read_csv('../input/ERY_df.txt' ,sep ='\t')# 目的変数
print('type of y is {}'.format(type(y_df)))
y_df.head()

for yy in range(y_df.shape[0]):
    if  y_df.iloc[yy, 0] == 'Positive':
         y_df.iloc[yy, 0] = 1
    else:
        y_df.iloc[yy, 0] = 0
        
y = y_df['ER_Status_nature2012'] #to Series
y.head()

type of y is <class 'pandas.core.frame.DataFrame'>


TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
Name: ER_Status_nature2012, dtype: int64

In [5]:
selected_col_num = 3

In [6]:
max_X_element = X.max(axis=1).max() 
min_X_element = X.min(axis=1).min() 

In [7]:
num_samples = X.shape[0]
num_cols = X.shape[1]

## 特徴量選択
 - select_cols, else_colsを決める

In [8]:
selector = SelectKBest(score_func=f_regression, k=selected_col_num) 
selector.fit(X, y)
mask = selector.get_support() 

In [9]:
select_cols = []
else_cols = []
for ii in range(len(mask)):
    if mask[ii] == 0:
        else_cols.append(X.columns[ii])
    else:
         select_cols.append(X.columns[ii])

select_cols

['CA12', 'AGR3', 'ESR1']

## すべての不等式が成り立つ：条件
したいこと→ハミルトニアンを最小化させる（xを最適化）

In [10]:
# W : ay - byの最大値

#xの要素で最大値とxの要素の最小値を知れば推定はできる
W = int((max_X_element *1*num_samples - min_X_element*1*num_samples)//1)

In [11]:
def make_y(y_before):
    select_index = random.randrange(num_samples)
    y_before[select_index] = 1 - y_before[select_index] 
    y_after = y_before
    return y_after

In [12]:
def make_ax_by(y_after):
    ax_by = []
    for i in range(selected_col_num):
        for j in range(num_cols - selected_col_num):
            ax_by.append(np.dot(X[select_cols[i]], y_after) - np.dot(X[else_cols[j]], y_after) +1)
    return ax_by 

In [13]:
def H(ax_by, index_ax_by):
    z = [0]*W
        
    each_ax_by = ax_by[index_ax_by]
    int_each_axby = each_ax_by //1
    sum_w_z = 0
    
    if int_each_axby >= 1:
        z[int(int_each_axby)] = 1
        sum_w_z = sum([int_each_axby*z[w] for w in range(W)])
    else:
        z[0] = 1
        sum_w_z = sum([1*z[w] for w in range(W)])
        
    sum_z = sum(z)
    z_series = pd.Series(z)

    ans_h =  (1-sum_z)**2 + (sum_w_z -  int(int_each_axby))**2
    return ans_h,  z_series

In [14]:
#know ori_H_sum
ax_by = make_ax_by(y)

ori_H_sum =  sum([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
ori_z_list =  [H(ax_by, index_ax_by)[1] for index_ax_by in range(len(ax_by))]
ori_z_df = pd.concat(ori_z_list, axis = 1)

print(ori_H_sum) #70464156.0
print([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
print(ori_z_df)

70464156.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [15]:
#find ans_y which H is as low as ori_y

y_after =  make_y(y)
ax_by = make_ax_by(y_after)

each_H_sum =  sum([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
each_z_list =  [H(ax_by, index_ax_by)[1] for index_ax_by in range(len(ax_by))]
each_z_df = pd.concat(each_z_list, axis = 1)


if each_H_sum <= ori_H_sum:
    print(each_H_sum)
    print([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
    print(each_z_df)
    print(y_after) 

70230447.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [17]:
#少なめloop
iii = 0
for ii in range(100):
    y_after =  make_y(y)
    ax_by = make_ax_by(y_after)

    a_H_sum =  sum([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
    a_z_list =  [H(ax_by, index_ax_by)[1] for index_ax_by in range(len(ax_by))]
    a_z_df = pd.concat(a_z_list, axis = 1)
    if a_H_sum <= ori_H_sum:
        iii += 1
        print(iii) #47個出力
        print(y_after)

1
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    1
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    1
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    1
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0D

8
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    1
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    0
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    1
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0D

15
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    1
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    0
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    1
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0

22
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    1
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    0
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    1
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0

29
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    0
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    0
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    1
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0

...

43
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    0
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    0
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    0
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0

50
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    0
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    0
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    0
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0

57
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    0
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    0
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    0
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0

64
TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
TCGA-D8-A1XU-01    1
TCGA-AR-A0TV-01    1
TCGA-D8-A1XV-01    1
TCGA-AN-A03Y-01    1
TCGA-AN-A04D-01    0
TCGA-A8-A08I-01    0
TCGA-BH-A0B3-01    0
TCGA-C8-A1HE-01    1
TCGA-A2-A0CW-01    1
TCGA-EW-A1OZ-01    1
TCGA-A1-A0SQ-01    0
TCGA-C8-A1HM-01    1
TCGA-BH-A1EO-01    1
TCGA-BH-A0B9-01    0
TCGA-D8-A1XG-01    1
TCGA-E2-A1BC-01    1
TCGA-A8-A08Z-01    1
TCGA-EW-A1P0-01    1
TCGA-A8-A06O-01    1
TCGA-D8-A1JG-01    0
TCGA-B6-A0IA-01    0
TCGA-AR-A1AY-01    0
TCGA-D8-A1XA-01    1
TCGA-E2-A15G-01    1
TCGA-GM-A2DH-01    0
                  ..
TCGA-A2-A0YK-01    1
TCGA-AR-A251-01    1
TCGA-A1-A0SG-01    1
TCGA-A8-A094-01    1
TCGA-BH-A0B7-01    1
TCGA-E2-A1LA-01    1
TCGA-A1-A0SM-01    1
TCGA-C8-A12Z-01    0
TCGA-AO-A0JD-01    1
TCGA-BH-A0BS-01    1
TCGA-E9-A22H-01    1
TCGA-A8-A0A1-01    1
TCGA-BH-A18I-01    1
TCGA-E2-A1IJ-01    1
TCGA-D8-A1X6-01    1
TCGA-B6-A0RU-01    0
TCGA-BH-A0

KeyboardInterrupt: 